### Checking for GPU availability

In [ ]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Runing on: {}'.format(device))

### Testing our U-Net based model implementation

In [ ]:
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
%matplotlib inline

from unet_model import *
from image_utils import *
from oral_mice_tissues_dataset import *

torch.cuda.empty_cache()

# load the input image (X) and its corresponding mask (Y)
dysplasia_level = "1-healthy"
color_normalization = "1-original"
X = load_pil_image("/home/dalifreire/Documents/Doutorado/github/histological_oral_mice_tissues/roi/image/train/{}/{}/image003-2-roi1.tif".format(color_normalization, dysplasia_level))
y = load_pil_image("/home/dalifreire/Documents/Doutorado/github/histological_oral_mice_tissues/roi/mask/{}/image003-2-roi1.tif".format(dysplasia_level))

X, y = data_augmentation(input_image=X, output_mask=y, aug=False)
X = X.unsqueeze(0).to(device)
y = y.unsqueeze(0).to(device)
print('X     --> {}'.format(X.size()))
print('y     --> {}'.format(y.size()))


# load our u-net based model
with torch.no_grad():
    model = UNet(in_channels=3, out_channels=1, padding=True).to(device)

criterion = nn.BCELoss().to(device)
optimizer = optim.Adam(model.parameters())
optimizer.zero_grad()       

# runs our u-net based model for the input image (X) and predicts the y_hat output mask
y_hat = model(X).squeeze(0)
y_hat = TF.to_tensor(np_to_pil(hysteresis_threshold(np_img=y_hat.detach().cpu().numpy()))) # hysteresis threshold
print('y_hat --> {}'.format(y_hat.size()))

# test the weight update step
loss = criterion(y_hat, y)
loss.backward()
optimizer.step()


# Show the output images
X_numpy = X.cpu().numpy()
y_numpy = y.cpu().numpy()
y_hat_numpy = y_hat.detach().cpu().numpy()

f, ax = plt.subplots(1, 3, figsize=(15, 15))
ax[0].imshow(X[0].cpu().permute(1, 2, 0))
ax[0].set_title('Input image')

ax[1].imshow(y_numpy[0], cmap='gray')
ax[1].set_title('Target mask')

ax[2].imshow(y_hat_numpy[0], cmap='gray')
ax[2].set_title('Output mask')

### Training our U-Net based model implementation

In [ ]:
from unet_model import *
from oral_mice_tissues_dataset import *

import math
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable


# create the datasets
batch_size = 1
color_normalization = "1-original"
dataset_dir = "/home/dalifreire/Documents/Doutorado/github/histological_oral_mice_tissues/roi"
dataloaders = create_dataloader(method=color_normalization, batch_size=batch_size, dataset_dir=dataset_dir)


# Checking for GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Runing on: {}'.format(device))


# load our u-net based model
torch.cuda.empty_cache()
model = UNet(in_channels=3, out_channels=1, padding=True).to(device)
criterion = nn.BCELoss().to(device)
optimizer = optim.Adam(model.parameters())


# train the model
since = time.time()
n_epochs = 500
model.train()
for epoch in range(1, n_epochs+1):

    if (epoch == 1 or epoch == n_epochs or epoch%50 == 0 ):
        print("")
        time_elapsed = time.time() - since
        print('Epoch {}/{} ({:.0f}m {:.0f}s)'.format(epoch, n_epochs, time_elapsed // 60, time_elapsed % 60))
        print('-' * 20)
    for batch_idx, (data, target, fname, original_size) in enumerate(dataloaders['train']):
        
        data = Variable(data.to(device))
        target = Variable(target.to(device))
        #print('X     --> {}'.format(data.size()))
        #print('y     --> {}'.format(target.size()))
                
        optimizer.zero_grad()
        #output = model(data)
        output = model(data).squeeze(0)
        output = TF.to_tensor(np_to_pil(hysteresis_threshold(np_img=y_hat.detach().cpu().numpy()))) # hysteresis threshold
        #print('y_hat --> {}'.format(output.size()))
        
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if (epoch == 1 or epoch == n_epochs or epoch%50 == 0 ) and ((batch_idx+1)%20 == 0):
            print('\tBatch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                (batch_idx+1), 
                (batch_idx+1) * len(data), 
                dataset_sizes['train'],
                100. *(((batch_idx+1) * len(data)) / dataset_sizes['train']), 
                loss.item()))
        if loss.item() < 0.001 or math.isnan(loss.item()):
            break
            
    #print("\tLoss: {:.6f}".format(loss.item()))
    if loss.item() < 0.001 or math.isnan(loss.item()):
        break
        
time_elapsed = time.time() - since
print('-' * 20)
print('{:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))


# Save the trained model
torch.save({
            'epoch': epoch,
            'batch_size': batch_size,
            'dataset': 'OralMiceTissuesDataset_{}'.format(color_normalization),
            'model_in_channels': model.model_input_channels(),
            'model_out_channels': model.model_output_channels(),
            'model_up_mode': model.model_up_mode(),
            'model_padding': model.model_padding(),
            'criterion': 'nn.BCELoss',
            'optimizer': 'optim.Adam',
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss
            }, "{}__Dataset-{}__Epoch-{}__Size-448x256.pt".format(model.name(), color_normalization, epoch))

### Runs our trained U-Net based model on test dataset

In [ ]:
from unet_model import *
from oral_mice_tissues_dataset import *


from torchvision import transforms
from torchvision import utils
from torch.autograd import Variable

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np


#model = load_checkpoint(file_path="{}__Dataset-{}__Epoch-{}__Size-448x256.pt".format(model.name(), color_normalization, epoch))


# create the datasets
batch_size = 1
color_normalization = "1-original"
dataset_dir = "/home/dalifreire/Documents/Doutorado/github/histological_oral_mice_tissues/roi"
dataloaders = create_dataloader(method=color_normalization, batch_size=batch_size, dataset_dir=dataset_dir)


# Checking for GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Runing on: {}'.format(device))


# directory to save results
results_dir = "results/{}".format(color_normalization)
if not os.path.exists(Path(results_dir)):
    os.makedirs(Path(results_dir))


# iterate over test dataset images        
for batch_idx, (images, masks, fname, original_size) in enumerate(dataloaders['test']):
    
    
    X = Variable(images.to(device))
    print('Batch {}: {}/{} images: {} masks: {}'.format(
        (batch_idx+1), 
        (batch_idx+1) * len(images), 
        len(dataloaders['test'].dataset),
        images.shape,
        masks.shape))
    
    
    y_hat = model(X).squeeze(0)
    X_numpy = X.cpu().numpy()
    y_hat_numpy = y_hat.detach().cpu().numpy()
    
    
    input_image_filename = '{}/{} - input.png'.format(results_dir, fname[0][0:-4])
    mask_image_filename = '{}/{} - mask.png'.format(results_dir, fname[0][0:-4])
    output_image_filename = '{}/{} - output.png'.format(results_dir, fname[0][0:-4])
    f, ax = plt.subplots(1, 3, figsize=(20, 20))
    
    
    # save the input image
    input_image = images[0]
    utils.save_image(input_image, input_image_filename)
    ax[0].imshow(input_image.permute(1, 2, 0))
    ax[0].set_title('{} (input)'.format(fname[0]))
    
    # save the mask image
    mask_image = masks[0]
    utils.save_image(output_image, output_image_filename)
    ax[1].imshow(mask_image, cmap='gray')
    ax[1].set_title('{} (output)'.format(fname[0]))
    
    # save the predicted output image
    output_image = y_hat[0]
    utils.save_image(output_image, output_image_filename)
    ax[2].imshow(output_image.squeeze(0).detach().cpu().numpy(), cmap='gray')
    ax[2].set_title('{} (output)'.format(fname[0]))
    